해당 노트북은 YouTube [todaycode오늘코드](https://www.youtube.com/channel/UCLR3sD0KB_dWpvcsrLP0aUg) 강의를 기반으로 작성했습니다.

# 1. 서울시에서 공개한 코로나19 발생동향 분석
* https://www.seoul.go.kr/coronaV/coronaStatus.do

* 11월 중순 이후 확진자가 증가함에 따라 '확진자 현황' 데이터를 여러개 페이지(1, 2, 3, ..)로 구분하여 웹사이트에 공개! -> 기존 pd.read_html()로 데이터 불러올 수 없음
* read_html()로는 칼럼명을 불러오고, 데이터는 requests 라이브러리로 읽어오는 방식 적용

In [1]:
# 필요한 라이브러리
import pandas as pd

In [2]:
# 크롤링으로 가져올 url 주소
url = "https://www.seoul.go.kr/coronaV/coronaStatus.do"
url

'https://www.seoul.go.kr/coronaV/coronaStatus.do'

In [3]:
pd.read_html?

Signature:
pd.read_html(
    io: Union[str, pathlib.Path, IO[~AnyStr]],
    match: Union[str, Pattern] = '.+',
    flavor: Union[str, NoneType] = None,
    header: Union[int, Sequence[int], NoneType] = None,
    index_col: Union[int, Sequence[int], NoneType] = None,
    skiprows: Union[int, Sequence[int], slice, NoneType] = None,
    attrs: Union[Dict[str, str], NoneType] = None,
    parse_dates: bool = False,
    thousands: Union[str, NoneType] = ',',
    encoding: Union[str, NoneType] = None,
    decimal: str = '.',
    converters: Union[Dict, NoneType] = None,
    na_values=None,
    keep_default_na: bool = True,
    displayed_only: bool = True,
) -> List[pandas.core.frame.DataFrame]
Docstring:
Read HTML tables into a ``list`` of ``DataFrame`` objects.

Parameters
----------
io : str, path object or file-like object
    A URL, a file-like object, or a raw string containing HTML. Note that
    lxml only accepts the http, ftp and file url protocols. If you have a
    URL that starts w

In [4]:
# read_html로 url 웹 사이트에서 별도의 로그인 없이 볼 수 있는 html의 table 태그(<tabel class ...)가 있는 데이터를 모두 불러옴!
table = pd.read_html(url)

In [5]:
# 테이블 개수는 잘 가져옴
len(table)

6

In [6]:
# 첫 번째 table: 구별 누적 확진자수 및 특정 시간 기준 추가 확진자수
table[0] # PC 화면용 테이블

,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구
0,508,360,510,582,579,842,983,992,616,727,1007,966,650
1,+2,+3,+3,+1,+6,+4,+3,+2,+1,+7,+2,+8,+1
2,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,기타
3,840,888,1543,767,384,861,991,1163,956,1129,1336,726,2891
4,+4,+2,+4,+6,+3,+4,0,+7,+4,0,+4,+4,+20


In [7]:
# 
table[1] # 모바일 화면용 테이블

,종로구,중구,용산구,성동구,광진구,동대문구
0,508,360,510,582,579,842
1,+2,+3,+3,+1,+6,+4
2,중랑구,성북구,강북구,도봉구,노원구,은평구
3,983,992,616,727,1007,966
4,+3,+2,+1,+7,+2,+8
5,서대문구,마포구,양천구,강서구,구로구,금천구
6,650,840,888,1543,767,384
7,+1,+4,+2,+4,+6,+3
8,영등포구,동작구,관악구,서초구,강남구,송파구
9,861,991,1163,956,1129,1336


In [8]:
# 11월 중순 확진자 증가로 인해 테이블 데이터를 페이지(1, 2, 3, 4, ...)를 나눠서 공개! -> read_html()로 불러올 수 없다!
df = table[3]
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황


## 11월 중순 이후 데이터 공개방식 변경
* [requests 라이브러리 설명](https://en.wikipedia.org/wiki/Requests_(software))

In [9]:
# requests: 작은 브라우저로 웹사이트를 읽어오는 목적!
# -> 웹사이트의 이미지, 텍스트, 표의 HTML 코드만 반환!
import requests 

* 1페이지 테이블에 대한 url을 알고 싶을 때, 테이블 우클릭 -> 검사 -> Network 탭 -> XHR -> 해당 draw에 맞는 get_status_ajax.php 선택 -> Headers -> Response URL 전체 주소 복사!
![uu](img/draw_1_URL.PNG)

In [10]:
# '1페이지 테이블'의 Request URL을 전체 복사한후, columns, start 기준으로 f-string으로 문자열 연결!
# get_status_ajax.php가 웹사이트 '확진자 현황' 탭 데이터를 가지고 있음!
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1"
url = f"{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc"
url = f"{url}&start=0&length=100&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1611812340714"
url

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&colum

In [11]:
# url 상태 확인(?): 200이면, 정상적으로 받아왔음을 의미!
response = requests.get(url)
response

<Response [200]>

In [12]:
# response.json(): '1페이지 테이블' 데이터 반환! 
data_json = response.json()
data_json

{'draw': 1,
 'recordsTotal': 13797,
 'recordsFiltered': 13797,
 'data': [["<p class='corona19_no'>23797</p>",
   '76876',
   '2021-01-27',
   '금천구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23796</p>",
   '76442',
   '2021-01-26',
   '타시도',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23795</p>",
   '76831',
   '2021-01-27',
   '영등포구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23794</p>",
   '76830',
   '2021-01-27',
   '영등포구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23793</p>",
   '76906',
   '2021-01-27',
   '도봉구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23792</p>",
   '76884',
   '2021-01-27',
   '관악구',
   '-',
   '강남구 소재 직장3 관련',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23791</p>",
   '76886',
   '2021-01-27',
   '강북구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23790</p>",
  

In [13]:
# 1페이지 중, url에 지정한 start=0&length=100만큼의 데이터만 가져왔지만, 웹사이트 1페이지~끝 페이지까지의 모든 데이터 개수도 알 수 있음
data_json["recordsTotal"]

13797

In [14]:
# 그중에 url에 지정한  에 의해 처음부터 100개 데이터만 가져옴! 
pd.DataFrame(data_json["data"])

,0,1,2,3,4,5,6
0,<p class='corona19_no'>23797</p>,76876,2021-01-27,금천구,-,기타 확진자 접촉,<b class=''></b>
1,<p class='corona19_no'>23796</p>,76442,2021-01-26,타시도,-,기타 확진자 접촉,<b class=''></b>
2,<p class='corona19_no'>23795</p>,76831,2021-01-27,영등포구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>23794</p>,76830,2021-01-27,영등포구,-,기타 확진자 접촉,<b class=''></b>
4,<p class='corona19_no'>23793</p>,76906,2021-01-27,도봉구,-,기타 확진자 접촉,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>23702</p>,76595,2021-01-27,서대문구,-,감염경로 조사중,<b class=''></b>
96,<p class='corona19_no'>23701</p>,76540,2021-01-27,도봉구,-,동대문구 소재 현대대중사우나 관련,<b class=''></b>
97,<p class='corona19_no'>23700</p>,76718,2021-01-27,관악구,-,기타 확진자 접촉,<b class=''></b>
98,<p class='corona19_no'>23699</p>,76577,2021-01-27,마포구,미국,해외유입,<b class=''></b>


In [15]:
# **위 특정 페이지 테이블의 Request URL이 전부 필요하지는 않고, draw, start(데이터 시작 인덱스 -> 1페이지가 10개 데이터씩 있다면, draw=2(2페이지) 데이터 일때는 start=10이어야 함!), length(한 페이지 데이터 길이)만 있어도 데이터를 가져올 수 있음

# draw: 웹사이트 페이지 번호를 누를때마다 1씩 증가!
# => draw=1: 최초 1페이지 테이블을 가져올 수 있음! 
url = "http://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1"
#url = url + "&order%5B0%5D%5Bdir%5D=desc&start=100&length=1000"

url = url + "&order%5B0%5D%5Bdir%5D=desc&start=0&length=100"



In [16]:
response = requests.get(url)
data_json = response.json()

In [17]:
data_json

{'draw': 1,
 'recordsTotal': 13797,
 'recordsFiltered': 13797,
 'data': [["<p class='corona19_no'>23797</p>",
   '76876',
   '2021-01-27',
   '금천구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23796</p>",
   '76442',
   '2021-01-26',
   '타시도',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23795</p>",
   '76831',
   '2021-01-27',
   '영등포구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23794</p>",
   '76830',
   '2021-01-27',
   '영등포구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23793</p>",
   '76906',
   '2021-01-27',
   '도봉구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23792</p>",
   '76884',
   '2021-01-27',
   '관악구',
   '-',
   '강남구 소재 직장3 관련',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23791</p>",
   '76886',
   '2021-01-27',
   '강북구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23790</p>",
  

In [18]:
# records_total: 웹페이지 1~끝페이지 데이터 개수
records_total = data_json["recordsTotal"]
# 끝 페이지 번호! -> 개략적인 계산을 위해 +1을 더해줬다는데.
end_page = round(records_total / 100) + 1 
end_page

139

In [19]:
data = data_json["data"]
data

[["<p class='corona19_no'>23797</p>",
  '76876',
  '2021-01-27',
  '금천구',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23796</p>",
  '76442',
  '2021-01-26',
  '타시도',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23795</p>",
  '76831',
  '2021-01-27',
  '영등포구',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23794</p>",
  '76830',
  '2021-01-27',
  '영등포구',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23793</p>",
  '76906',
  '2021-01-27',
  '도봉구',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23792</p>",
  '76884',
  '2021-01-27',
  '관악구',
  '-',
  '강남구 소재 직장3 관련',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23791</p>",
  '76886',
  '2021-01-27',
  '강북구',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23790</p>",
  '76878',
  '2021-01-27',
  '성동구',
  '-',
  '타시도 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>23789</p>",
  '

In [20]:
# 0 ~ 6까지 칼럼명이 없음
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>23797</p>,76876,2021-01-27,금천구,-,기타 확진자 접촉,<b class=''></b>
1,<p class='corona19_no'>23796</p>,76442,2021-01-26,타시도,-,기타 확진자 접촉,<b class=''></b>
2,<p class='corona19_no'>23795</p>,76831,2021-01-27,영등포구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>23794</p>,76830,2021-01-27,영등포구,-,기타 확진자 접촉,<b class=''></b>
4,<p class='corona19_no'>23793</p>,76906,2021-01-27,도봉구,-,기타 확진자 접촉,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>23702</p>,76595,2021-01-27,서대문구,-,감염경로 조사중,<b class=''></b>
96,<p class='corona19_no'>23701</p>,76540,2021-01-27,도봉구,-,동대문구 소재 현대대중사우나 관련,<b class=''></b>
97,<p class='corona19_no'>23700</p>,76718,2021-01-27,관악구,-,기타 확진자 접촉,<b class=''></b>
98,<p class='corona19_no'>23699</p>,76577,2021-01-27,마포구,미국,해외유입,<b class=''></b>


## 위 기능과 동일하게 1페이지만 가져오는 함수 정의

In [29]:
def get_seoul_covid19_100(page_no):
    
    start_no = (page_no - 1) * 100
    
    url = f"http://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_no}&length=100"
    
    response = requests.get(url)
    data_json = response.json()
    return data_json

In [34]:
# 1페이지 데이터
get_seoul_covid19_100(1)

{'draw': 1,
 'recordsTotal': 13797,
 'recordsFiltered': 13797,
 'data': [["<p class='corona19_no'>23797</p>",
   '76876',
   '2021-01-27',
   '금천구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23796</p>",
   '76442',
   '2021-01-26',
   '타시도',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23795</p>",
   '76831',
   '2021-01-27',
   '영등포구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23794</p>",
   '76830',
   '2021-01-27',
   '영등포구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23793</p>",
   '76906',
   '2021-01-27',
   '도봉구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23792</p>",
   '76884',
   '2021-01-27',
   '관악구',
   '-',
   '강남구 소재 직장3 관련',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23791</p>",
   '76886',
   '2021-01-27',
   '강북구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23790</p>",
  

In [24]:
# 5페이지 데이터
get_seoul_covid19_100(5)

{'draw': 5,
 'recordsTotal': 13797,
 'recordsFiltered': 13797,
 'data': [["<p class='corona19_no'>23397</p>",
   '75281',
   '2021-01-24',
   '영등포구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23396</p>",
   '75247',
   '2021-01-24',
   '성북구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23395</p>",
   '75288',
   '2021-01-24',
   '타시도',
   '-',
   '종로구 소재 빌딩관련',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23394</p>",
   '75235',
   '2021-01-24',
   '동작구',
   '-',
   '타시도 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23393</p>",
   '75336',
   '2021-01-24',
   '서초구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23392</p>",
   '75177',
   '2021-01-24',
   '구로구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23391</p>",
   '75270',
   '2021-01-24',
   '은평구',
   '-',
   '동부구치소 관련',
   "<b class=''></b>"],
  ["<p class='corona19_no'>23390</p>",
   '7524

In [23]:
# 존재하지 않는 페이지수 입력 시, 데이터 없음!
get_seoul_covid19_100(1000)

{'draw': 1000, 'recordsTotal': 13797, 'recordsFiltered': 13797, 'data': []}

## 전체 데이터 가져오기
tqdm 사용!
* 오래 걸리는 작업의 진행상태 표시


In [26]:
# time.sleep으로 시차를 두기 위함
import time

# tqdm: 진행 상태 표시를 위함 
from tqdm import trange

* 1) 일단 1~3페이지만 데이터가 제대로 로드되는지 확인 
    * 처음부터 전체 페이지를 수집하면, 중간에 오류가 나도 찾기 어려움
    * 일부만 우선 가져와 보고, 잘 동작하면 나머지 전체 가져오기!

In [30]:

page_list = []

all_page = 3
for page_no in trange(all_page+1): # for ~ in range()처럼 all_page + 1을 해준다! 
    one_page_data_json = get_seoul_covid19_100(page_no)
    one_page_df = pd.DataFrame(one_page_data_json["data"])
    page_list.append(one_page_df)
    
    # 서버에 한 번에 너무 많은 요청을 보내면 부담이 되므로.. 
    # 0.5초씩 쉬었다가 크롤링 수행
    time.sleep(0.5)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.39it/s]


In [31]:
# 3페이지 데이터까지 잘 가져옴!
pd.concat(page_list)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>23797</p>,76876,2021-01-27,금천구,-,기타 확진자 접촉,<b class=''></b>
1,<p class='corona19_no'>23796</p>,76442,2021-01-26,타시도,-,기타 확진자 접촉,<b class=''></b>
2,<p class='corona19_no'>23795</p>,76831,2021-01-27,영등포구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>23794</p>,76830,2021-01-27,영등포구,-,기타 확진자 접촉,<b class=''></b>
4,<p class='corona19_no'>23793</p>,76906,2021-01-27,도봉구,-,기타 확진자 접촉,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>23502</p>,75653,2021-01-25,중구,-,기타 확진자 접촉,<b class='status1'></b>
96,<p class='corona19_no'>23501</p>,75645,2021-01-25,도봉구,-,도봉구 소재 어린이집 관련,<b class='status1'></b>
97,<p class='corona19_no'>23500</p>,75716,2021-01-25,종로구,-,종로구 소재 요양시설2 관련,<b class='status1'></b>
98,<p class='corona19_no'>23499</p>,75684,2021-01-25,기타,-,은평구 소재 병원1 관련,<b class='status1'></b>


* 2) 전체 페이지 데이터 로드

In [72]:
def get_multi_page_list(start_page, end_page=80):
    
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page_data_json = get_seoul_covid19_100(page_no)
        
        if len(one_page_data_json["data"]) > 0:
            one_page_df = pd.DataFrame(one_page_data_json["data"])
            page_list.append(one_page_df)
            time.sleep(0.5)
        else:
            # 수집된 값이 없으면, page_list를 곧바로 반환
            return page_list
    
    return page_list

In [73]:
start_page = 1
page_list = get_multi_page_list(start_page, end_page) # 2021.01.28 기준 1페이지 ~ end_page(139) 페이지까지의 데이터 크롤링
# 데이터가 많아서 1페이지 데이터만 확인
page_list[:1]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 138/139 [01:59<00:00,  1.16it/s]


[                                   0      1           2     3   4  \
 0   <p class='corona19_no'>23797</p>  76876  2021-01-27   금천구   -   
 1   <p class='corona19_no'>23796</p>  76442  2021-01-26   타시도   -   
 2   <p class='corona19_no'>23795</p>  76831  2021-01-27  영등포구   -   
 3   <p class='corona19_no'>23794</p>  76830  2021-01-27  영등포구   -   
 4   <p class='corona19_no'>23793</p>  76906  2021-01-27   도봉구   -   
 ..                               ...    ...         ...   ...  ..   
 95  <p class='corona19_no'>23702</p>  76595  2021-01-27  서대문구   -   
 96  <p class='corona19_no'>23701</p>  76540  2021-01-27   도봉구   -   
 97  <p class='corona19_no'>23700</p>  76718  2021-01-27   관악구   -   
 98  <p class='corona19_no'>23699</p>  76577  2021-01-27   마포구  미국   
 99  <p class='corona19_no'>23698</p>  76553  2021-01-27  동대문구   -   
 
                      5                 6  
 0            기타 확진자 접촉  <b class=''></b>  
 1            기타 확진자 접촉  <b class=''></b>  
 2            기타 확진자 접촉  <

In [74]:
df_all = pd.concat(page_list)
df_all.shape

(13797, 7)

In [78]:
df_all.head(1)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>23797</p>,76876,2021-01-27,금천구,-,기타 확진자 접촉,<b class=''></b>


In [84]:
# read_html()로 읽어온 table[3]의 칼럼명을 리스트로 만들어서 df_all에 적용
df_all.columns = table[3].columns.tolist()
df_all.head(1)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>23797</p>,76876,2021-01-27,금천구,-,기타 확진자 접촉,<b class=''></b>


연번, 퇴원현황 칼럼 데이터 전처리가 필요해보임

## 데이터 전처리 (연번, 퇴원현황 칼럼)
* re 라이브러리 사용